## PDF 전처리

- [소득세법](https://law.go.kr/lsSc.do?section=&menuId=1&subMenuId=15&tabMenuId=81&eventGubun=060101&query=%EC%86%8C%EB%93%9D%EC%84%B8%EB%B2%95#liBgcolor17) pdf 을 가지고 LangChain 의 Document 로 전처리해보기
- PDF 를 파싱한 내용들 중 표, 서식, 이미지와 같은 것들이 제대로 가져왔는지를 확인해야한다.
- 참고 자료
    - [LangChain 의 공식 문서](https://python.langchain.com/docs/how_to/document_loader_pdf/)

In [ ]:
import os


# tokenizers 내부 병렬화 유지 + 경고 억제; docling 에서 ocr 사용시의 warning 해결
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [1]:
from pathlib import Path


PDF_FILE_PATH = Path("../data/income-tax-law.pdf")

### Docling 을 사용한 parsing

- [공식 홈페이지](https://docling-project.github.io/docling/)
- [Docling GitHub](https://github.com/docling-project/docling)
- RAG 를 위한 PDF 파싱 라이브러리로 자세한 내용은 [이 문서](/docs/what-is-docling.md)를 참고
- [LangChain 용 Docling 문서](https://python.langchain.com/docs/integrations/document_loaders/docling/#load)

- DoclingLoader는 “변환 → 직렬화 →(선택) 청킹”을 한 번에 처리하는 편의 래퍼
    - 변환은 문서를 구조화 모델(DoclingDocument)로 만드는 과정
    - 직렬화는 그 구조화 모델을 Markdown/JSON 등 최종 포맷으로 내보내는 과정
    - 청킹은 문서를 쪼개는 과정

### 기본 설정으로 파싱

In [2]:
from langchain_docling import DoclingLoader


default_loader = DoclingLoader(file_path=PDF_FILE_PATH)
default_docs = default_loader.load()

print("총 Docling Document 개수:", len(default_docs))
default_docs[0]

2025-09-20 15:43:06,658 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-09-20 15:43:06,682 - INFO - Going to convert document batch...
2025-09-20 15:43:06,683 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e647edf348883bed75367b22fbe60347
2025-09-20 15:43:06,691 - WARNING - The plugin langchain_docling will not be loaded because Docling is being executed with allow_external_plugins=false.
2025-09-20 15:43:06,691 - INFO - Loading plugin 'docling_defaults'
2025-09-20 15:43:06,693 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-09-20 15:43:06,701 - WARNING - The plugin langchain_docling will not be loaded because Docling is being executed with allow_external_plugins=false.
2025-09-20 15:43:06,701 - INFO - Loading plugin 'docling_defaults'
2025-09-20 15:43:06,709 - INFO - Registered ocr engines: ['easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-09-20 15:43:14,706 - INFO - Accelerator device: 'mps'
2025-09-20 15:43:16,566 - I

총 Docling Document 개수: 2493


Document(metadata={'source': PosixPath('../data/income-tax-law.pdf'), 'dl_meta': {'schema_name': 'docling_core.transforms.chunker.DocMeta', 'version': '1.0.0', 'doc_items': [{'self_ref': '#/texts/1', 'parent': {'$ref': '#/body'}, 'children': [], 'content_layer': 'body', 'label': 'text', 'prov': [{'page_no': 1, 'bbox': {'l': 30.0, 't': 660.18, 'r': 202.16, 'b': 651.203, 'coord_origin': 'BOTTOMLEFT'}, 'charspan': [0, 25]}]}, {'self_ref': '#/texts/2', 'parent': {'$ref': '#/body'}, 'children': [], 'content_layer': 'body', 'label': 'text', 'prov': [{'page_no': 1, 'bbox': {'l': 34.0, 't': 634.18, 'r': 564.72, 'b': 609.203, 'coord_origin': 'BOTTOMLEFT'}, 'charspan': [0, 105]}]}, {'self_ref': '#/texts/3', 'parent': {'$ref': '#/body'}, 'children': [], 'content_layer': 'body', 'label': 'text', 'prov': [{'page_no': 1, 'bbox': {'l': 50.0, 't': 602.18, 'r': 157.15, 'b': 593.203, 'coord_origin': 'BOTTOMLEFT'}, 'charspan': [0, 20]}]}], 'origin': {'mimetype': 'application/pdf', 'binary_hash': 10046112

### 커스텀 설정으로 파싱

- 아래의 코드를 사용하기 위해선 시스템에 [tesseract](https://github.com/tesseract-ocr/tesseract) 와 한글, 한문 언어팩을 설치해야한다.

In [4]:
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions, TesseractCliOcrOptions
from docling.document_converter import DocumentConverter, PdfFormatOption
from langchain_docling.loader import ExportType


# 1) 파이프라인 옵션: OCR + 테이블 구조 인식

pdf_opts = PdfPipelineOptions(
    do_table_structure=True,
    do_ocr=True,
)

# 2) OCR 엔진과 언어 설정(한국어+간체중국어)
pdf_opts.ocr_options = TesseractCliOcrOptions(lang=["kor", "chi_sim"])

# (표 정확도 향상) 셀 매칭 활성화
pdf_opts.table_structure_options.do_cell_matching = True

custom_loader = DoclingLoader(
    file_path=PDF_FILE_PATH,
    export_type=ExportType.MARKDOWN,
    converter=DocumentConverter(
        allowed_formats=[InputFormat.PDF],
        format_options={
            InputFormat.PDF: PdfFormatOption(pipeline_options=pdf_opts),
        },
    ),
)
custom_docs = custom_loader.load()

print("총 Docling Document 개수:", len(custom_docs))

2025-09-20 15:44:43,960 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-09-20 15:44:43,966 - INFO - Going to convert document batch...
2025-09-20 15:44:43,967 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 24bcc513fad89c13641d7803793186d2
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2025-09-20 15:44:44,046 - INFO - command: tesseract --list-langs
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2025-09-20 

총 Docling Document 개수: 1


In [6]:
out_path = Path("../data/income-tax-law-parsed-by-docling.md")
with out_path.open("w", encoding="utf-8") as f:
    f.write(custom_docs[0].page_content)